In [ ]:
# 1. Install and import required packages
!pip install earthengine-api geemap --quiet

import ee
import geemap
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 2. Authenticate and initialize
ee.Authenticate()
ee.Initialize(project='ee-salempar')  # Your Earth Engine project

# 3. Define the region of interest (ROI) from your asset
roi = ee.FeatureCollection("projects/ee-salempar/assets/Kinshasa_AiresSelect2")

# 4. Define date range
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 12, 31)

# 5. Export function
def export_era5_rasters(band_name, label_prefix, scale, folder, reducer):
    current = start_date
    while current <= end_date:
        next_month = current + relativedelta(months=1)
        label = current.strftime("%Y_%m")

        # Define time range for filtering
        ee_start = ee.Date(current.strftime("%Y-%m-%d"))
        ee_end = ee.Date(next_month.strftime("%Y-%m-%d"))

        # Filter and reduce
        image = (ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
                 .filterDate(ee_start, ee_end)
                 .filterBounds(roi)
                 .select(band_name)
                 .reduce(reducer)
                 .clip(roi))

        # Convert temperature to Celsius if needed
        if band_name == 'temperature_2m':
            image = image.subtract(273.15).rename(band_name)

        # Set export parameters
        task = ee.batch.Export.image.toDrive(**{
            'image': image,
            'description': f"{label_prefix}_{label}",
            'folder': folder,
            'fileNamePrefix': f"{label_prefix}_{label}",
            'scale': scale,
            'region': roi.geometry(),
            'crs': 'EPSG:4326',
            'maxPixels': 1e13
        })
        task.start()
        print(f"✅ Started export: {label_prefix}_{label}")

        current = next_month

# 6. Run exports
export_era5_rasters('temperature_2m', 'ERA5_Temp', 1000, 'GEE_Kinshasa', ee.Reducer.mean())
export_era5_rasters('total_precipitation_sum', 'ERA5_Precip', 1000, 'GEE_Kinshasa', ee.Reducer.sum())
